In [1]:
"""
This model has been consider as complete.
That means it should not be changed, if is required, create a new copy and modify it.
"""

In [2]:
# Import Gurobi

from gurobipy import *
import numpy as np

In [3]:
# Create the model

M = Model(name = 'PI 1');

Academic license - for non-commercial use only


In [4]:
# Team names

names = ['Independiente Medellín', 'Atlético Nacional',
         'Deportivo Cali',         'América de Cali',
         'La Equidad',             'Patriotas',
         'Envigado F.C',           'Rionegro Águilas',
         'Once Caldas',            'Deportivo Pasto',
         'Unión Magdalena',        'Junior',
         'Atlético Huila',         'Deportes Tolima',
         'Jaguares',               'Alianza Petrolera',
         'Atlético Bucaramanga',   'Cúcuta Deportivo',
         'Millonarios',            'Santa Fe']

In [5]:
# Model parameters

# Number of teams
n = 20
T = [i for i in range(n)]

# Number of dates
m = 30
P = [i for i in range(m)]

# Date of non classical matches
c = 15
Q = [i for i in range(m) if i != c]

# Team-pairs that share stadium
S = [(0, 1), (2, 3), (18, 19)]

# Classic matches
C = [(i, i + 1) for i in range(0, n - 1, 2)]

# Prohibited dates for some teams (due to other competitions)
F = {

    # Copa sudamericana
    2 : [21, 29],               # Deportivo Cali
    4 : [17, 25],               # La Equidad
    7 : [17, 25],               # Rionegro Águilas
    8 : [5, 9],                 # Once Caldas

    # Libertadores
    0  : [5, 7],                # Independiente Medellín
    1  : [5, 7, 9, 11],         # Atlético Nacional
    11 : [13, 15, 21, 23, 27],  # Junior
    13 : [13, 15, 21, 23, 27],  # Deportes Tolima

    # Copa Colombia
    3  : [7, 15, 18, 23, 27],   # América de Cali
    5  : [7, 15, 18, 23, 27],   # Patriotas
    6  : [7, 15, 18, 23, 27],   # Envigado
    9  : [7, 15, 18, 23, 27],   # Deportivo Pasto
    10 : [15, 18, 21, 23, 27],  # Unión Magdalena
    12 : [7, 15, 18, 23, 27],   # Atlético Huila
    14 : [7, 15, 18, 23, 27],   # Jaguares
    15 : [7, 15, 18, 23, 27],   # Alianza Petrolera
    16 : [7, 15, 18, 23, 27],   # Atlético Bucaramanga
    17 : [7, 15, 18, 23, 27],   # Cúcuta Deportivo
    18 : [7, 15, 18, 23, 27],   # Millonarios
    19 : [7, 15, 18, 23, 27]    # Santa Fe
}

F2 = F.copy()
for key in F2.keys():
    dates = np.zeros(m)
    for value in F2[key]:
        dates[value - 1] = 1
    F2[key] = dates

# L_jk = 1 if team j plays as visitant in the date k in
# an external tournament (Copa Libertadores, Sudamericana, etc.)
L = {

    # Libertadores
    0  : [5],          # Independiente Medellín
    1  : [5, 9],       # Atlético Nacional
    11 : [15, 21, 23], # Junior
    13 : [15, 23],     # Deportes Tolima

    # Sudamericana
    2 : [29],          # Deportivo Cali
    4 : [17],          # La Equidad
    7 : [25],          # Rionegro
    8 : [5],           # Once Caldas
}

for i in range(n):
    if not i in L.keys():
        L[i] = []

for key in L.keys():
    dates = np.zeros(m)
    for value in L[key]:
        dates[value - 1] = 1
    L[key] = dates

# Best teams (based on 2019 1)
# Millonarios, Cali, Tolima, América, Nacional, Pasto, Junior & Unión Magdalena
b = 8
B = [18, 2, 13, 3, 1, 9, 11, 10]

# Worst teams (based on 2019 1)
# Santa Fe, Rionegro, Huila, Bucaramanga, Equidad, Jaguares, Alianza Petrolera, Envigado
w = 8
W = [19, 7, 12, 16, 4, 14, 15, 6]

In [6]:
# Inizializate the variables

# X_ijk = 1 if team i plays against team j in date k
X = []

for i in T:
    row = []
    for j in T:
        row.append([None] * m)
    X.append(row)

for i in T:
    for j in T:
        for k in P:
            vname = '{} vs {} in date {}'.format(names[i], names[j], k + 1)
            X[i][j][k] = M.addVar(vtype = GRB.BINARY, name = vname)

# Decision variable (Y_jk = 1 if team j must play as visitor in date k and k + 1)
Y = [None] * n
for i in range(n):
    row = [None] * (m - 1)
    for k in range(m - 1):
        row[k] = M.addVar(lb = 0, name = 'y_{} {}'.format(i, k), vtype = GRB.BINARY)
    Y[i] = row

# Artificial variable
phi = M.addVar(lb = 0, vtype = GRB.INTEGER);

In [7]:
# Restrictions

"""
1. All teams play once against each other
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for k in Q) == 1 for i in T for j in T if i != j))

"""
2. Each team must play the same number of matches as local as visitor
"""
M.addConstrs((quicksum(X[i][j][k] - X[j][i][k] for j in T if j != i for k in P) == 0 for i in T));

"""
3. Each team can play at most one match each date
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for j in T) + F2[i][k] <= 1 for i in T for k in P));

"""
4. Teams that share stadium cannot play as local the same date
"""
M.addConstrs((quicksum(X[i][h][k] + X[j][h][k] for h in T) <= 1 for i, j in S for k in P));

"""
5. Classic matches must be played only once
"""
M.addConstrs((X[i][j][c] + X[j][i][c] == 1 for i, j in C));

"""
6. Classic matches can not be repeated
"""
M.addConstrs((quicksum(X[i][j][k] for k in Q) + X[i][j][c] == 1 for i, j in C));

"""
7. Teams can not play in the forbidden dates
"""
M.addConstrs((quicksum(X[i][j][k - 1] + X[j][i][k - 1] for k in F[i]) == 0 for i in T for j in T if i != j));

In [8]:
# New restrictions [September 5]

"""
8. If Y_jk is 1, the team j must play as visitant in dates k and k + 1
"""
M.addConstrs((quicksum(X[i][j][k] + X[i][j][k + 1] for i in T) + L[j][k] + L[j][k + 1] <= 1 + Y[j][k] for j in T for k in P[:-1]));

"""
9. No more than phi dates as visitant for any team
"""
M.addConstrs((quicksum(Y[j][k] for k in P[:-1]) <= phi for j in T));

"""
10. No more than 3 dates as visitor for any team
"""
M.addConstrs((quicksum(quicksum(X[i][j][k1] for i in T) + L[j][k1] for k1 in range(k, k + 3)) <= 2 for j in T for k in P[:-2]));

In [9]:
# New restrictions [September 12]

"""
11. Half of the matches must be played before classic date
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for k in P[:c+1] for j in T) == n // 2 for i in T));

"""
12. Best teams
"""
M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) == b // 2 for i in T if not i in B));

M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) <= b // 2 for i in B));

M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) >= (b // 2) - 1 for i in B));

"""
13. Worst teams
"""
M.addConstrs((quicksum(X[i][j][k] for j in W if j != i for k in Q) == w // 2 for i in T if not i in W));

M.addConstrs(quicksum(X[i][j][k] for j in W if j != i for k in Q) <= w // 2 for i in W);

M.addConstrs(quicksum(X[i][j][k] for j in W if j != i for k in Q) >= (w // 2) - 1 for i in W);

In [10]:
# New restrictions [September 19]

"""
14. Classic matches can't be played in the first 6 dates of the tournament
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[7:]) == 2 for i, j in C);

"""
15. Classic matches must have at least 4 dates between them
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[c - 4 : c + 5]) == 1 for i, j in C);

"""
16. Half of the matches against best teams must be played before classical week 8
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) == b // 2 for j in T if not j in B);

M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) <= b // 2 for j in B);

M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) >= (b // 2) - 1 for j in B);

In [11]:
# Set an artificial objective

obj = phi
# M.setObjective(obj, GRB.MINIMIZE)

In [12]:
# Search for a feasible solution

M.setParam(GRB.Param.TimeLimit, 60);
M.optimize();

Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 2774 rows, 12581 columns and 164996 nonzeros
Variable types: 0 continuous, 12581 integer (12580 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1118 rows and 3718 columns
Presolve time: 0.76s
Presolved: 1656 rows, 8863 columns, 83827 nonzeros
Variable types: 0 continuous, 8863 integer (8862 binary)

Root relaxation: objective 0.000000e+00, 4789 iterations, 1.83 seconds
Total elapsed time = 6.30s
Total elapsed time = 11.01s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  401          -    0.00000      -     -   13s
     0     0    0.00000    0  469          -    0.00000      -     -   15s
     0     

In [13]:
# Print dates

print()
for variable in M.getVars():
    if variable.X == 1 and 'y' in variable.getAttr('varName'):
        print(variable.getAttr('varName') + "\t" + str(variable.X))

for variable in M.getVars():
    if variable.X == 1 and 'Millonarios' in variable.getAttr('varName'):
        print(variable.getAttr('varName'))

print('\nPhi:', phi.X)


Atlético Nacional vs Millonarios in date 20
Deportivo Cali vs Millonarios in date 1
Envigado F.C vs Millonarios in date 14
Deportivo Pasto vs Millonarios in date 22
Junior vs Millonarios in date 26
Atlético Huila vs Millonarios in date 12
Jaguares vs Millonarios in date 5
Alianza Petrolera vs Millonarios in date 10
Cúcuta Deportivo vs Millonarios in date 29
Millonarios vs Independiente Medellín in date 17
Millonarios vs América de Cali in date 21
Millonarios vs La Equidad in date 19
Millonarios vs Patriotas in date 24
Millonarios vs Rionegro Águilas in date 30
Millonarios vs Once Caldas in date 3
Millonarios vs Unión Magdalena in date 2
Millonarios vs Deportes Tolima in date 8
Millonarios vs Atlético Bucaramanga in date 9
Millonarios vs Santa Fe in date 28
Santa Fe vs Millonarios in date 16

Phi: -0.0


In [32]:
# Make files

import codecs

dats = []
for _ in T:
    row = []
    for k in P:
        row.append('')
    dats.append(row)

for i in T:
    for j in T:
        for k in P:
            if i != j and X[i][j][k].X == 1:
                dats[i][k] = 'L'
                dats[j][k] = 'V'
            if F2[i][k] == 1:
                if [3, 5, 6, 9, 10, 12, 14, 15, 16, 17, 18, 19].count(i) == 0:
                    dats[i][k] = 'FL'
                    if L[i][k] == 1:
                        dats[i][k] = 'FV'
                #else:
                #   dats[i][k] = 'C'


file = open('dates.txt', 'w')
for row in dats:

    file.write(','.join(row) + '\n')

file.close()

file2 = codecs.open('names.txt', 'w', 'utf-8')
file2.write('\n'.join(names) + '\n')
file2.close()
print(names)

['Independiente Medellín', 'Atlético Nacional', 'Deportivo Cali', 'América de Cali', 'La Equidad', 'Patriotas', 'Envigado F.C', 'Rionegro Águilas', 'Once Caldas', 'Deportivo Pasto', 'Unión Magdalena', 'Junior', 'Atlético Huila', 'Deportes Tolima', 'Jaguares', 'Alianza Petrolera', 'Atlético Bucaramanga', 'Cúcuta Deportivo', 'Millonarios', 'Santa Fe']
